# A MUSIC GENERATOR

Can music be generated by a computer? 

This was a question that came into my mind recently. From the knowledge and past research I had, this seemed possible. 
My initial thought process was that it was simply a matter of passing in music data, and getting a model to do the prediction by using some form of memorisation and classification (to predict the notes to come in the music socre).



However, one problem i immediatedly identified was how to convert(encode) music data and pass it through a neural network?

After all in cases of NLP it usually involves one hot encoding, and we can siply use numpy (RGB) for Computer Vision related problems, so how can i do so for audio data?

Upon further research i chanced upon multiple articles that had already tackled this problem. Namely, MIDI is largely used to prepare data to be fed into neural networks for such problems (Methods involving CV are definitly possible, but MIDI seems to give better results).

This mini project is a result of just some minor tweaks to the code and guides i dicovered online. 

IT IS NOT MEANT TO BE MY WORK, BUT RATHER AN ECPLORATION INTO THIS FIELD, AND AN EXPERIMENTATION INTO POSSIBLE AREAS OF IMPROVEMENT. 

Largely inspired via:
https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5

AND

https://www.analyticsvidhya.com/blog/2020/01/how-to-perform-automatic-music-generation/

Import the standard libraries

In [ ]:
import os
import numpy as np

Get datasets via google drive

In [ ]:
# mount  google drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
base_folder = '/content/drive/MyDrive' # Change this path to your folder for this competition
training_path = os.path.join(base_folder, "adl-piano-midi.zip") # zip file for training dataset
training_path_2 = os.path.join(base_folder, "midi_audio_data_dataset2.zip") # zip file for training dataset

In [ ]:
!unzip $training_path

Streaming output truncated to the last 5000 lines.
   creating: adl-piano-midi/Classical/Icelandic Classical/Ólafur Arnalds/
  inflating: adl-piano-midi/Classical/Icelandic Classical/Ólafur Arnalds/0040.mid  
   creating: adl-piano-midi/Classical/Classical Era/
   creating: adl-piano-midi/Classical/Classical Era/Muzio Clementi/
  inflating: adl-piano-midi/Classical/Classical Era/Muzio Clementi/Sonatina-1-1.mid  
  inflating: adl-piano-midi/Classical/Classical Era/Muzio Clementi/Sonatina-1.mid  
  inflating: adl-piano-midi/Classical/Classical Era/Muzio Clementi/Andante in B Flat.mid  
  inflating: adl-piano-midi/Classical/Classical Era/Muzio Clementi/Allegro.mid  
  inflating: adl-piano-midi/Classical/Classical Era/Muzio Clementi/Sonatina-1-2.mid  
  inflating: adl-piano-midi/Classical/Classical Era/Muzio Clementi/Sonatine no1.mid  
   creating: adl-piano-midi/Classical/Classical Era/Johann Baptist Wanhal/
  inflating: adl-piano-midi/Classical/Classical Era/Johann Baptist Wanhal/Rondo.m

In [ ]:
!unzip $training_path_2

Archive:  /content/drive/MyDrive/midi_audio_data_dataset2.zip
  inflating: albeniz/alb_esp1.mid    
  inflating: albeniz/alb_esp2.mid    
  inflating: albeniz/alb_esp3.mid    
  inflating: albeniz/alb_esp4.mid    
  inflating: albeniz/alb_esp5.mid    
  inflating: albeniz/alb_esp6.mid    
  inflating: albeniz/alb_se1.mid     
  inflating: albeniz/alb_se2.mid     
  inflating: albeniz/alb_se3.mid     
  inflating: albeniz/alb_se4.mid     
  inflating: albeniz/alb_se5.mid     
  inflating: albeniz/alb_se6.mid     
  inflating: albeniz/alb_se7.mid     
  inflating: albeniz/alb_se8.mid     
  inflating: bach/bach_846.mid       
  inflating: bach/bach_847.mid       
  inflating: bach/bach_850.mid       
  inflating: balakir/islamei.mid     
  inflating: beeth/appass_1.mid      
  inflating: beeth/appass_2.mid      
  inflating: beeth/appass_3.mid      
  inflating: beeth/beethoven_hammerklavier_1.mid  
  inflating: beeth/beethoven_hammerklavier_2.mid  
  inflating: beeth/beethoven_hammerkla

Music 21 is a Python library for understanding music data.

MIDI is a format for storing music files. 
MIDI files contain the instructions rather than the actual audio. Hence, it occupies very little memory.

In [ ]:
#library for understanding music
from music21 import *

Function to get notes from a midi file 


( Extract notes from the music, and break down the chords into notes:[ A chord is made iup of a few notes] )

In [ ]:
#defining function to read MIDI files
def read_midi(file):  
    notes=[]

    # parse file ie. read the file as a midi data format
    midi = converter.parse(file)
    notes_to_parse = None

    # Split it via instrument parts
    parts = instrument.partitionByInstrument(midi)

    if parts: # file has instrument parts
        notes_to_parse = parts.parts[0].recurse()
    else: # file has notes in a flat structure      #ie. theres only 1 instrument type.
        notes_to_parse = midi.flat.notes

    for element in notes_to_parse:
        if isinstance(element, note.Note):        #an instance or a note
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):    # an instance of a chord. A chord consist of a few note. Thats why we need to extract the notes out via for loop
            notes.append('.'.join(str(n) for n in element.normalOrder))
    return notes

Load the MIDI files

In [ ]:
#specify the path. Just get classical for now
path='/content/adl-piano-midi/Classical'

# #read all the filenames
# # goes into each folders, subfolder, and subSubFolders ect... to get the mmid files out
fileName = []
for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith(".mid"):
          fileName.append(root+'/'+file)

fileName = fileName[0:600]  # one of the files has some byte error (Corrupted midi file). How tragic.

In [ ]:
path_array = ['/content/albeniz', "/content/bach", "/content/balakir", "/content/beeth", "/content/borodin", "/content/brahms", "/content/burgm", "/content/chopin",
              "/content/debussy", "/content/granados", "/content/grieg", "/content/haydn", "/content/liszt", "/content/mendelssohn", "/content/mozart", "/content/muss", 
              "/content/schubert", "/content/schumann", "/content/tschai"]

def getdata(path):
  for root, dirs, files in os.walk(path):
      for file in files:
          if file.endswith(".mid"):
            fileName.append(root+'/'+file)

In [ ]:
for i in path_array:
  getdata(i)

In [ ]:
# reading each midi file
notes_array = np.array([read_midi(i) for i in fileName])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [ ]:
#converting 2D array into 1D array
notes_ = [element for note_ in notes_array for element in note_]

#No. of unique notes
unique_notes = list(set(notes_))
print(len(unique_notes))

#importing library
from collections import Counter

#computing frequency of each note
freq = dict(Counter(notes_))

661


We want to keep the notes that appear the most frequently. 

This enables us to arrow down the number of notes to predict (ie. reduce the numbers of classes for classification) (This of course helps in our prediction precess, as if there are too many classes the probability would be too spread out and we would get meaningless predictions)

"count >= xx" is basically a parameter that can be changed. In this case it has been set to 30, meaning we ignore notes that appear less than 30 times

In [ ]:
frequent_notes = [note_ for note_, count in freq.items() if count>=30]
print(len(frequent_notes))

289


In [ ]:
print(freq)
print(frequent_notes)

{'D4': 22467, 'D3': 14083, 'E4': 20584, 'F4': 18934, 'B-3': 16032, 'G4': 24421, 'A3': 19140, 'A4': 22942, 'G3': 20680, 'B4': 19392, 'C5': 26102, 'D5': 25357, '2.4': 945, '3.5': 588, 'E5': 20884, 'B3': 15932, '9.2': 2471, '7.0': 3108, 'F3': 15687, 'G#3': 15049, '1.5': 2394, '4.7': 4486, '6.9': 3676, '8.11': 2919, '9.1': 2777, 'E3': 13892, 'F#4': 14304, 'F#3': 10766, 'C4': 22928, 'C#5': 15618, 'E-5': 14918, 'F#5': 11979, 'E-4': 15104, 'G5': 18589, 'B2': 7684, 'C#4': 13947, '2.5': 4501, '0.4': 4369, '5.9': 3405, '5.7': 1016, '5.10': 2074, '3.7': 3434, 'A5': 13659, 'B5': 7495, 'C6': 8714, 'E6': 3550, 'G#5': 9723, 'G#4': 15371, 'F5': 18072, 'B-5': 8191, 'B-4': 16912, 'C#6': 4822, 'D6': 6239, '4.9': 2597, '2.6': 3333, '2.8': 1077, '6.10': 1617, '8.10': 617, '11.4': 2300, 'C3': 13362, 'A2': 8539, 'G2': 8121, 'F2': 5790, 'E2': 4078, 'D2': 3493, 'C2': 3843, 'E-3': 12185, 'B1': 1756, 'C#2': 3099, '4.8': 2502, 'A1': 1434, '0.2.5': 186, '7.9': 897, '0.5': 2260, '9.0': 4739, '7.11': 3730, '11.2': 4

Prepare musical files such that only the top frequent notes appear

In [ ]:
new_music=[]

for notes in notes_array:
    temp=[]
    for note_ in notes:
        if note_ in frequent_notes:
            temp.append(note_)            
    new_music.append(temp)
    
new_music = np.array(new_music)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # Remove the CWD from sys.path while we load stuff.


#Training Data preparation

In [ ]:
no_of_timesteps = 32

# x is the input
# y is the output
x = []
y = []

for note_ in new_music:
    for i in range(0, len(note_) - no_of_timesteps, 1):
        
        #preparing input and output sequences
        input_ = note_[i:i + no_of_timesteps]   # since timestep has been set to 32. it means that 32 notes would be used to predict the next note
        output = note_[i + no_of_timesteps]   # the 32 notes would be used to predict this 1 note
        
        x.append(input_)
        y.append(output)
        
x=np.array(x)
y=np.array(y)

Assign a unique integer to every note:

In [ ]:
unique_x = list(set(x.ravel()))
x_note_to_int = dict((note_, number) for number, note_ in enumerate(unique_x))

Prepare the integer sequences for input data

In [ ]:
#preparing input sequences
x_seq=[]
for i in x:
    temp=[]
    for j in i:
        #assigning unique integer to every note
        temp.append(x_note_to_int[j])
    x_seq.append(temp)
    
x_seq = np.array(x_seq)

Prepare the integer sequences for output data as well

In [ ]:
unique_y = list(set(y))
y_note_to_int = dict((note_, number) for number, note_ in enumerate(unique_y)) 
y_seq=np.array([y_note_to_int[i] for i in y])

Split 80% of the data for training and the rest 20% for testing

In [ ]:
from sklearn.model_selection import train_test_split
x_tr, x_val, y_tr, y_val = train_test_split(x_seq,y_seq,test_size=0.2,random_state=0)

#Build model (LSTM model)

In [ ]:
from keras.layers import *
from keras.models import *
from keras.callbacks import *
import keras.backend as K

K.clear_session()
model = Sequential()
    
#embedding layer
model.add(Embedding(len(unique_x), 100, input_length=32,trainable=True)) 

model.add(Conv1D(64,3, padding='causal',activation='relu'))
model.add(Dropout(0.2))
model.add(MaxPool1D(2))
    
model.add(Conv1D(128,3,activation='relu',dilation_rate=2,padding='causal'))
model.add(Dropout(0.2))
model.add(MaxPool1D(2))

model.add(Conv1D(256,3,activation='relu',dilation_rate=4,padding='causal'))
model.add(Dropout(0.2))
model.add(MaxPool1D(2))
          
#model.add(Conv1D(256,5,activation='relu'))    
model.add(GlobalMaxPool1D())
    
model.add(Dense(256, activation='relu'))
model.add(Dense(len(unique_y), activation='softmax'))
    
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 32, 100)           28900     
                                                                 
 conv1d (Conv1D)             (None, 32, 64)            19264     
                                                                 
 dropout (Dropout)           (None, 32, 64)            0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 16, 64)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 16, 128)           24704     
                                                                 
 dropout_1 (Dropout)         (None, 16, 128)           0         
                                                        

In [ ]:
mc=ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True,verbose=1)

In [ ]:
history = model.fit(np.array(x_tr),np.array(y_tr),batch_size=128,epochs=80, validation_data=(np.array(x_val),np.array(y_val)),verbose=1, callbacks=[mc])

Epoch 1/80
5758/5758 [==============================] - ETA: 0s - loss: 3.8417
Epoch 00001: val_loss improved from inf to 3.73444, saving model to best_model.h5
5758/5758 [==============================] - 91s 14ms/step - loss: 3.8417 - val_loss: 3.7344
Epoch 2/80
5755/5758 [============================>.] - ETA: 0s - loss: 3.6125
Epoch 00002: val_loss improved from 3.73444 to 3.63414, saving model to best_model.h5
5758/5758 [==============================] - 77s 13ms/step - loss: 3.6125 - val_loss: 3.6341
Epoch 3/80
5756/5758 [============================>.] - ETA: 0s - loss: 3.5496
Epoch 00003: val_loss improved from 3.63414 to 3.56116, saving model to best_model.h5
5758/5758 [==============================] - 77s 13ms/step - loss: 3.5496 - val_loss: 3.5612
Epoch 4/80
5757/5758 [============================>.] - ETA: 0s - loss: 3.5160
Epoch 00004: val_loss improved from 3.56116 to 3.52677, saving model to best_model.h5
5758/5758 [==============================] - 81s 14ms/step - loss

In [ ]:
#loading best model
from keras.models import load_model
model = load_model('best_model.h5')

Compose music

In [ ]:
import random
ind = np.random.randint(0,len(x_val)-1)

random_music = x_val[ind]

predictions=[]
for i in range(15):

    random_music = random_music.reshape(1,no_of_timesteps)

    prob  = model.predict(random_music)[0]
    y_pred= np.argmax(prob,axis=0)
    predictions.append(y_pred)

    random_music = np.insert(random_music[0],len(random_music[0]),y_pred)
    random_music = random_music[1:]
    
print(predictions)

[86, 86, 3, 86, 95, 3, 50, 50, 154, 154, 154, 220, 220, 220, 220]


Convert int back into notes

In [ ]:
x_int_to_note = dict((number, note_) for number, note_ in enumerate(unique_x)) 
predicted_notes = [x_int_to_note[i] for i in predictions]

convert predictions into a MIDI file

In [ ]:
def convert_to_midi(prediction_output):
   
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                
                cn=int(current_note)
                new_note = note.Note(cn)
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
                
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
            
        # pattern is a note
        else:
            
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 1
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='music.mid')

In [ ]:
convert_to_midi(predicted_notes)